In [3]:
from system.flcore.trainmodel.resnet import *
import torch
model = resnet18_low(num_classes=10, has_bn=True, bn_block_num=4).to("cuda:0")
model.fc = torch.nn.Identity()
print(model)

ResNetwithLowPrototype(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer_0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer_1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(

In [4]:
import torchvision
import torchvision.transforms as transforms
raw_data = 'data/Cifar10/'
transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(
        root=raw_data+"rawdata", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(
root=raw_data+"rawdata", train=False, download=True, transform=transform)
    
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=len(trainset.data), shuffle=False, drop_last=True)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=len(testset.data), shuffle=False, drop_last=True)

100%|██████████| 170498071/170498071 [00:03<00:00, 55357846.33it/s]


Extracting data/Cifar10/rawdata/cifar-10-python.tar.gz to data/Cifar10/rawdata
Files already downloaded and verified


In [5]:
def agg_func(protos):
    for [label, proto_list] in protos.items():
        if len(proto_list) > 1:
            proto = 0 * proto_list[0].data
            for i in proto_list:
                proto += i.data
            protos[label] = proto / len(proto_list)
        else:
            protos[label] = proto_list[0] #只有一个，不需要平均了

    return protos #不一定包含所有种类

def save_embedding(protos, is_high=True):
    protos = torch.tensor(protos)
    if is_high: protos.save("high.pt")
    else: protos.save("low.pt")

In [7]:
model.eval()
from collections import defaultdict
import torch.nn.functional as F
import tqdm

protos = defaultdict(list)
device = "cuda:0"
low_protos = defaultdict(list)
with torch.no_grad():
    for i, (x, y) in enumerate(trainloader):
        if type(x) == type([]):
            x[0] = x[0].to(device)
        else:
            x = x.to(device)
        y = y.to(device)
        low, rep = model(x)
        rep = F.normalize(rep, dim=1) #[batch, dim(512，对于resnet18)]
        low = F.normalize(low, dim=1)
        for i, yy in enumerate(y):
            y_c = yy.item()
            protos[y_c].append(rep[i, :].detach().data)
            low_protos[y_c].append(low[i, :].detach().data)
        
protos = agg_func(protos)
low_protos = agg_func(low_protos)

In [14]:
print(protos.items)
print(low_protos)

<built-in method items of collections.defaultdict object at 0x7faea1ad1940>
defaultdict(<class 'list'>, {6: tensor([[[0.0303, 0.0341, 0.0293,  ..., 0.0294, 0.0301, 0.0782],
         [0.0300, 0.0278, 0.0243,  ..., 0.0252, 0.0233, 0.0524],
         [0.0160, 0.0264, 0.0213,  ..., 0.0233, 0.0239, 0.0760],
         ...,
         [0.0153, 0.0252, 0.0215,  ..., 0.0185, 0.0226, 0.0841],
         [0.0229, 0.0350, 0.0248,  ..., 0.0188, 0.0285, 0.0743],
         [0.0687, 0.0832, 0.0758,  ..., 0.0620, 0.0682, 0.1158]],

        [[0.0473, 0.0649, 0.0770,  ..., 0.0721, 0.1126, 0.1371],
         [0.0592, 0.0724, 0.0731,  ..., 0.0678, 0.0906, 0.1115],
         [0.0357, 0.0647, 0.0698,  ..., 0.0621, 0.0818, 0.1187],
         ...,
         [0.0252, 0.0592, 0.0610,  ..., 0.0584, 0.0805, 0.1239],
         [0.0343, 0.0706, 0.0728,  ..., 0.0659, 0.0732, 0.1260],
         [0.0663, 0.1007, 0.1108,  ..., 0.1084, 0.1149, 0.1580]],

        [[0.0363, 0.0208, 0.0226,  ..., 0.0206, 0.0292, 0.0145],
         [0.026